Impress your friends and co-workers with your ability to provide answers to the New York Time's, Spelling Bee word game! 

For the uninitated the NYTSB is a daily word puzzel where you get 7 letters, 6 outer and 1 inner letters - You score points by making words from the available letters! Each word must include the center letter and be longer than 3 characters. In this notebook and supporting files you will see 3 solutions.

The first method is a brute force attempt.

# List of Words

The code does not know which combination of letters consitute a word. Utilized Natural Lanugage Processing Tool Kit (NLTK)

# Beginner

### The Simple Son: Compare each valid character combination from available letters to a list of words.

For the first two solutions the user had to manually enter in the puzzel characters.

ImportError: cannot import name 'method1' from 'spelling_bee' (/Users/drahcir1/Documents/GitHub/NYT_Spelling_bee/spelling_bee.py)

In [4]:
Pro()


NameError: name 'Pro' is not defined

In [6]:
!pwd


/Users/drahcir1/Documents/GitHub/NYT_Spelling_bee
